## **Fixed charge location problem**


finding a balance between benefit of covering demands and cost of building facilities.


**_MIP Model_**


**_Indices and Parameters_**

-   $I = \{1, 2, ..., m\}$: A set of demand points.
-   $J = \{1, 2, ..., n\}$: A set of potential facility locations.
-   $d_{ij}$: unit shipping cost of $i$ to location $j$.
-   $f_{j}$: building cost at location $j$.
-   $h_{j}$: demand size of $i$


**_Decision Variables_**

-   $x_j$: Binary variable that equals 1 if a facility is allocated at location $j$ and 0 otherwise, for $j \in J$.
-   $y_{i,j}$: Binary variable that equals 1 if a demand $i$ is covered by $i$


**_Objective Function_**

Minimize Total Cost:

$$
\text{Minimize} \quad Z = \sum_{j \in J} f_j x_j + \sum_{i \in I} \sum_{j \in J} d_{ij} h_i y_{ij}
$$

1. **Demand Fulfillment Constraint**:
   Each demand point must be served by exactly one facility.

    $$
    \sum_{j \in J} y_{ij} = 1 \quad \forall i \in I
    $$

2. **Facility Allocation Constraint**:
   A demand point \(i\) can only be served by a facility \(j\) if that facility is actually established.

    $$
    y_{ij} \leq x_j \quad \forall i \in I, \forall j \in J
    $$

3. **Binary Conditions**:
   The decision variables must be binary.
    $$
    x_j \in \{0,1\} \quad \forall j \in J
    $$
    $$
    y_{ij} \in \{0,1\} \quad \forall i \in I, \forall j \in J
    $$


**_Code_**


In [29]:
import pulp
class FixedChargeFLP:
    def __init__(self, I, J, f, h, d):
        self.I = I
        self.J = J
        self.f = f
        self.h = h
        self.d = d

        # Initialize the problem
        self.problem = pulp.LpProblem("FixedChargeFLP", pulp.LpMinimize)

        #DVs
        self.x = pulp.LpVariable.dicts("x", self.J, cat = pulp.LpBinary)
        self.y = pulp.LpVariable.dicts("y", [(i,j) for i in self.I for j in self.J], cat = pulp.LpBinary)

    def model(self):
        #obj
        self.problem += pulp.lpSum(self.x[j]*self.f[j] for j in self.J)
        + pulp.lpSum(self.d[(i,j)] * self.y[(i,j)] * self.h[i] for i in self.I for j in self.J)

        #constraints:
        #1: Each demand point must be served by exactly one facility.
        for i in self.I:
            self.problem += pulp.lpSum(self.y[(i,j)] for j in self.J) == 1
        
        #2: Facility Allocation Constraint
        for i in self.I:
            for j in self.J:
                self.problem += self.y[(i,j)] <= self.x[j]

    def solve(self):
        self.problem.solve()
        print(f"Status: {pulp.LpStatus[self.problem.status]}")
        if self.problem.status == pulp.LpStatusOptimal:
            print(f"Optimal value: {pulp.value(self.problem.objective)}")

        for j in self.J:
            print(f"x[{j}] = {self.x[j].varValue}")
        for i in self.I:
            for j in self.J:
                print(f"y[{i},{j}] = {self.y[(i,j)].varValue}")
        #print Xj
        for j in self.J:
            if self.x[j].value() == 1:
                print(f"Facility at location {j} is allocated.")
        
        #printing yij
        for i in self.I:
            for j in self.J:
                if self.y[(i,j)].value() == 1:
                    print(f"demand {i} is covered by facility {j}")

        



In [30]:
# Example data
I = [1, 2]  # Demand points
J = [1, 2, 3]  # Facility locations
f = {1: 1000, 2: 1500, 3: 2000}  # Fixed costs
h = {1: 10, 2: 20}  # Demand sizes
d = {(1, 1): 4, (1, 2): 6, (1, 3): 8, (2, 1): 10, (2, 2): 12, (2, 3): 14}  # Shipping costs

flp = FixedChargeFLP(I, J, f, h, d)
flp.model()
flp.solve()
for j in J:
    print(f"x[{j}] = {flp.x[j].varValue}")
for i in I:
    for j in J:
        print(f"y[{i},{j}] = {flp.y[(i,j)].varValue}")

Status: Optimal
Optimal value: 1000.0
x[1] = 1.0
x[2] = 0.0
x[3] = 0.0
y[1,1] = 1.0
y[1,2] = 0.0
y[1,3] = 0.0
y[2,1] = 1.0
y[2,2] = 0.0
y[2,3] = 0.0
Facility at location 1 is allocated.
demand 1 is covered by facility 1
demand 2 is covered by facility 1


NameError: name 'self' is not defined